In [1]:
import numpy as np
import pandas as pd 
import os
from numpy import float32
from xgboost import XGBClassifier
import gc
import pickle
from sklearn.externals import joblib

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
promoted_content = pd.read_csv('promoted_content.csv')
document_category = pd.read_csv('documents_categories.csv')
document_topic = pd.read_csv('documents_topics.csv')
document_entity = pd.read_csv('documents_entities.csv')
events_data = pd.read_csv('events.csv')
clicks_train  = pd.read_csv('clicks_train.csv')
clicks_test = pd.read_csv('clicks_test.csv')

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
clicks_test = pd.read_csv('clicks_test.csv')

In [4]:
events_data = pd.read_csv('events.csv')

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
clicks_train.shape

(87141731, 3)

In [6]:
clicks_test.shape

(32225162, 2)

In [7]:
def splitDataFrameIntoSmaller(df, chunkSize = 15000000):  ## change variable names..
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

In [8]:
split_train_frame = splitDataFrameIntoSmaller(clicks_train)

In [9]:
chunk_frame = split_train_frame[0]

In [10]:
chunk_frame.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


# Events - File 0

In [11]:
merged_df_with_Event = pd.DataFrame(events_data)

In [12]:
merged_df_with_Event.drop(['uuid','timestamp','document_id'],axis=1,inplace=True)

In [13]:
merged_df_with_Event['Country']=merged_df_with_Event['geo_location'].str[0:2]

In [14]:
merged_df_with_Event.drop(['geo_location'],axis=1,inplace=True)

In [15]:
merged_df_with_Events = pd.merge(chunk_frame,merged_df_with_Event,how='left',on=(['display_id']))

In [16]:
merged_df_with_Events['Country'] = pd.Categorical.from_array(merged_df_with_Events.Country).labels
merged_df_with_Events = merged_df_with_Events.query("Country == Country")

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  if __name__ == '__main__':
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  if __name__ == '__main__':


In [17]:
merged_df_with_Events['platform'] = pd.Categorical.from_array(merged_df_with_Events.platform).labels
merged_df_with_Events['platform'] = pd.to_numeric(merged_df_with_Events['platform'], errors='ignore')

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: Categorical.from_array is deprecated, use Categorical instead
  if __name__ == '__main__':
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  if __name__ == '__main__':


In [18]:
merged_df_with_Events.dtypes

display_id    int64
ad_id         int64
clicked       int64
platform       int8
Country       int16
dtype: object

# Promoted Content - File 1

In [19]:
merged_df = pd.merge(merged_df_with_Events,promoted_content,how='left',on='ad_id')

# Document Category - File 2

In [20]:
average_category_confidenceLevel = pd.DataFrame(document_category.groupby(['document_id'])['confidence_level'].mean()).reset_index()

In [21]:
merged_df_with_CL_Cat = pd.merge(merged_df,average_category_confidenceLevel,how='left',on='document_id')

In [22]:
merged_df_with_CL_Cat = merged_df_with_CL_Cat[np.isfinite(merged_df_with_CL_Cat['confidence_level'])]

In [23]:
merged_df_with_CL_Cat['confidence_level'] = merged_df_with_CL_Cat['confidence_level'].map('{:,.3f}'.format)

In [24]:
merged_df_with_CL_Cat[['confidence_level']] = merged_df_with_CL_Cat[['confidence_level']].astype(float)

# Document Topics - File 3

In [25]:
average_topic_confidenceLevel = pd.DataFrame(document_topic.groupby(['document_id'])['confidence_level'].mean()).reset_index()

In [26]:
merged_df_with_CL_Top = pd.merge(merged_df_with_CL_Cat,average_topic_confidenceLevel,how='left',on='document_id')

In [27]:
merged_df_with_CL_Top = merged_df_with_CL_Top[np.isfinite(merged_df_with_CL_Top['confidence_level_y'])]

In [28]:
merged_df_with_CL_Top['confidence_level_y'] = merged_df_with_CL_Top['confidence_level_y'].map('{:,.3f}'.format)

In [29]:
merged_df_with_CL_Top[['confidence_level_y']] = merged_df_with_CL_Top[['confidence_level_y']].astype(float)

# Document Entities - File 4

In [30]:
average_entity_confidenceLevel = pd.DataFrame(document_entity.groupby(['document_id'])['confidence_level'].mean()).reset_index()

In [31]:
merged_df_with_CL_Ent = pd.merge(merged_df_with_CL_Top,average_entity_confidenceLevel,how='left',on='document_id')

In [32]:
merged_df_with_CL_Ent.to_csv("merged_df_with_CL_Ent.csv")

In [35]:
gc.collect()

108

In [37]:
merged_df_with_CL_Ent = pd.read_csv("merged_df_with_CL_Ent.csv")

MemoryError: 

In [ ]:
merged_df_with_CL_Ent = merged_df_with_CL_Ent[np.isfinite(merged_df_with_CL_Ent['confidence_level'])]

In [ ]:
merged_df_with_CL_Ent['confidence_level'] = merged_df_with_CL_Ent['confidence_level'].map('{:,.3f}'.format)

In [ ]:
merged_df_with_CL_Ent[['confidence_level']] = merged_df_with_CL_Ent[['confidence_level']].astype(float)

In [ ]:
merged_df_with_CL_Ent[['platform']] = merged_df_with_CL_Ent[['platform']].astype(int)

In [ ]:
merged_df_with_CL_Ent.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
merged_df_with_CL_Ent.shape

In [ ]:
merged_df_with_CL_Ent.to_csv("Document_Metadata.csv", index=False)

In [ ]:
document_metadata = pd.read_csv("Document_Metadata.csv")

# Model Creation

In [ ]:
X = [x for x in merged_df_with_CL_Ent.columns if x not in ['display_id','clicked']]

In [ ]:
Y = ['clicked']

In [ ]:
model = XGBClassifier(max_depth=3,n_estimators=400, learning_rate=0.02)

In [ ]:
train_model = model.fit(merged_df_with_CL_Ent[X],merged_df_with_CL_Ent[Y])

# Save Trained Model

In [ ]:
save_model = pickle.dumps(train_model)

In [ ]:
load_model = pickle.loads(save_model)

In [ ]:
joblib.dump(train_model, 'xgboost.pkl') 

In [ ]:
trained_model = joblib.load('xgboost.pkl') 

In [ ]:
split_test_frame = splitDataFrameIntoSmaller(clicks_test)

In [ ]:
test = split_test_frame[0]

In [ ]:
predY=[]

In [ ]:
chunk_test = pd.merge(test,merged_df_with_Events,how='left',on='display_id')

In [ ]:
chunk_test_promo = pd.merge(chunk_test,promoted_content,how='left',on='ad_id')

In [ ]:
chunk_test_with_CL_Cat = pd.merge(chunk_test_promo,average_category_confidenceLevel,how='left',on='document_id')
chunk_test_with_CL_Cat = chunk_test_with_CL_Cat.fillna(0.0)

In [ ]:
chunk_test_with_CL_Top = pd.merge(chunk_test_with_CL_Cat,average_topic_confidenceLevel,how='left',on='document_id')
chunk_test_with_CL_Top=chunk_test_with_CL_Top.fillna(0.0)

In [ ]:
chunk_test_with_CL_Ent = pd.merge(chunk_test_with_CL_Top,average_entity_confidenceLevel,how='left',on='document_id')
chunk_test_with_CL_Ent=chunk_test_with_CL_Ent.fillna(0.0)

In [ ]:
X_test = [x for x in chunk_test_with_CL_Ent.columns if x not in ['display_id','clicked']]

In [ ]:
chunk_pred=list(trained_model.predict_proba(chunk_test_with_CL_Ent[X_test]).astype(float)[:,1])

In [ ]:
predY += chunk_pred

In [ ]:
results=pd.concat((clicks_test,pd.DataFrame(predY)) ,axis=1,ignore_index=True)#Combine the predicted values with the ids

In [ ]:
results.columns = ['display_id','ad_id','clicked']#Rename the columns

In [ ]:
results.to_csv("xgb_results.csv", index=False)

In [ ]:
results_1 = pd.read_csv('xgb_results.csv')

In [ ]:
results_1 = results_1.sort_values(by=['display_id','clicked'], ascending=[True, False])

In [ ]:
results_1.head()

In [ ]:
final_file = results_1.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [ ]:
final_file.head()

In [ ]:
final_file.to_csv("XGB_File_400trees_final.csv", index=False)

### Some other check points

train_data.to_csv("train_data.csv",index=False)
train_data.head()

In [ ]:
train_data = chunk_test_with_CL_Ent[X_test]
train_data.head()

In [ ]:
train_data[document_id == 1051283]